## Yearly BBH merger detection calculation

import input below

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate
from scipy import integrate
from pesummary.io import read
from astropy.cosmology import LambdaCDM
import astropy.units as u
# for waveform generator
#import logging
#import io
import bilby

%matplotlib inline
%config InlineBackend.figure_format='retina'


cosmo = LambdaCDM(H0=70, Om0=0.3, Ode0=0.7)
R0    = 30           #rate scale- no./GPc^3 /year
Mmin  = 5
Mmax  = 50


#################### P(w) data.  ####################
Pw    = np.loadtxt('pw_detectors_data/pw_single.txt')
w     = Pw[:,0]


#################### SENSITIVITY ####################
ligoO3 = np.loadtxt('aligo_O3actual_H1.txt')
psd3   = (ligoO3[:,1] )**2      
fligo3 = ligoO3[:,0]


############# Previously generated Pdet from UNIFORM m & z ####################
pdet12 = np.load('DATA/Pdet_N1200uniform.npy')    # for N = 1200 x 1200 etc.
pdet10 = np.load('DATA/Pdet_N1000uniform.npy')
pdet8  = np.load('DATA/Pdet_N800uniform.npy')
pdet6  = np.load('DATA/Pdet_N600uniform.npy')
pdet4  = np.load('DATA/Pdet_N400uniform.npy')
pdet2  = np.load('DATA/Pdet_N200uniform.npy')

#################### Pdet from random m and z #################### 
pdet16ran = np.load('DATA/Pdet_N1600.npy')
pdet14ran = np.load('DATA/Pdet_N1400.npy')
pdet12ran = np.load('DATA/Pdet_N1200.npy')
pdet8ran  = np.load('DATA/Pdet_N800rand.npy')
pdet6ran  = np.load('DATA/Pdet_N600.npy')
pdet4ran  = np.load('DATA/Pdet_N400.npy')
pdet2ran  = np.load('DATA/Pdet_N200.npy')

#################### Randomly generated m and z #################### 
mgen16  = np.load('DATA/mmatrixN1600_NoOrder.npy')
mgen14  = np.load('DATA/mmatrixN1400_NoOrder.npy')
mgen12  = np.load('DATA/mmatrixN1200_NoOrder.npy')
mgen8   = np.load('DATA/mmatrixN800_NoOrder.npy')

zgen12  = np.load('DATA/zmatrixN1200_NoOrder.npy')
zgen14  = np.load('DATA/zmatrixN1400_NoOrder.npy')
zgen16  = np.load('DATA/zmatrixN1600_NoOrder.npy')
zgen8   = np.load('DATA/zmatrixN800_NoOrder.npy')


## Functions
### i.e. Waveform generator

In [6]:
def z_distribution(sample_num, alpha,beta,z_p,zmax):
    #originally alpha = 1.6, beta = 3.4, z_p = 2.4 #can change these to input if needed

    z      = np.arange(0,zmax,0.1) #model range
    frac   = (1+z)/(1+z_p)
    pz     = ((1+z)**alpha ) / (1+ frac**(alpha+beta) ) #distribution

    CDFz   = integrate.cumtrapz(pz,z)
    Cz     = CDFz[-1]
    nCDFz  = CDFz/Cz
    
    sample = np.random.uniform( nCDFz[0],1,sample_num)
    intpZ  = interpolate.interp1d( nCDFz,z[:-1] ) #x y
    newz   = intpZ(sample)
    return newz, Cz


def m_distribution(sample_num,mmin,mmax,a):
    m  = np.arange(mmin,mmax,0.1)
    pm = m**(a)      # the distribution 
    
    CDFm = integrate.cumtrapz(pm,m)
    Cm = CDFm[-1]
    nCDFm = CDFm/Cm
    
    sample = np.random.uniform(0.01,1,sample_num)
    intpm = interpolate.interp1d( nCDFm,m[:-1] ) #x y
    newm = intpm(sample)
    return newm, Cm


def tiling(m,z,ns):
    mt = np.tile( m, (ns,1) )
    mT = np.transpose( mt )
    
    zt = np.tile( z,(ns,1) )
    return mT,zt

# Function for mass ratio q = m2/m1, not used here
"""
def qdist(mmin,m1,a,sample_num): 
    #we first pick the larger one, M1.
    q = np.arange(mmin/m1,1.2,0.1) #mmin/m1

    phi = q**(-a) #a=0.1
    
    CDFm = integrate.cumtrapz(phi,q)
    Cm   = CDFm[-1]
    nCDFm = CDFm/Cm #new normalised phi cumulative
    
    sample = np.random.uniform( nCDFm[0],1, sample_num ) 
    #interpolate(y,x)
    intpm = interpolate.interp1d( nCDFm,q[:-1])#, bounds_error=False, fill_value=(mmin/m1,1) ) #fill value with qmin qmax
    newq  = intpm(sample)
    
    m2 = m1*newq                  # conts / array len samplenum

    return newq,m2#,m2[(m2>5)&(m2<50)]

qdist = np.vectorize(qdist)
"""


def pwfunc(wdata):
    f = interpolate.interp1d( w, Pw[:,1],bounds_error=False,fill_value=(1,0) ) 
    newPw = f(wdata)
    return newPw


def Fp(RA,DEC,PSI): 
    phi = RA
    theta = np.pi/2.0 - DEC
    return 0.5*(1 + np.cos(theta)**2)*np.cos(2*phi)*np.cos(2*PSI) - np.cos(theta)*np.sin(2*phi)*np.sin(2*PSI)

#psi=polarisation angle
def Fx(RA,DEC,PSI): 
    phi = RA
    theta = np.pi/2.0 - DEC
    return 0.5*(1 + np.cos(theta)**2)*np.cos(2*phi)*np.sin(2*PSI) + np.cos(theta)*np.sin(2*phi)*np.cos(2*PSI)




duration = 2.0
sampling_frequency = 2048.0
##### WAVEFORM ARGUMENTS #####
waveform_arguments = dict(
        waveform_approximant = 'IMRphenomXAS', #'IMRPhenomXPHM', #IMRphenomXAS
        reference_frequency = 50.,
        minimum_frequency = 170, #20.,
        maximum_frequency = 1000, #2000.,
        mode_array = [[2,2],[2,-2]]#,[2,1],[2,-1],[3,3],[3,-3],[3,2],[3,-2],[4,4],[4,-4]]
    )

waveform_generator = bilby.gw.WaveformGenerator(duration = duration,
    sampling_frequency = sampling_frequency,
    frequency_domain_source_model = bilby.gw.source.lal_binary_black_hole,
    parameter_conversion = bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters,
    waveform_arguments = waveform_arguments,
    )

    
    
def waveform(Binary_parameters, Waveform_arguments,Fx,Fp):
    gen_h = waveform_generator.frequency_domain_strain(Binary_parameters)
    fs = bilby.core.utils.series.create_frequency_series(sampling_frequency,duration)

    hp = gen_h['plus']
    hx = gen_h['cross']
    
    # MAXIMUM SETTING 
    #h = Fx*hx + Fp*hp
    hmod2 = np.conj(hp)*hp
    return hx, hp, hmod2, fs


psdinterp3    = interpolate.interp1d( fligo3, psd3, bounds_error=False, fill_value=(1,1) )

def SNR(fs, hmod2, psdinterp ):
    #f    = interpolate.interp1d( PSDf, PSDs, bounds_error=False, fill_value=(1,1) ) #fill_value when Snf out of range to 1 to make func go to 0
    inte = hmod2 / psdinterp(fs) 
    return np.sqrt( 4*integrate.trapezoid(inte,fs ) )


   
    
def Pdet( m1,Z,m2,psdinterp ): 
    Ra  = 0  
    Dec = 0 
    Psi = 0
    LD = cosmo.luminosity_distance(Z) 
    binary_parameters = dict(
                mass_1= m1*(1+Z),
                mass_2= m2*(1+Z),
                a_1 = 0 ,
                a_2 = 0,
                tilt_1= 0,                      
                tilt_2= 0,
                phi_12= 0,
                phi_jl= 0,
                luminosity_distance = LD/u.Mpc , # Mpc 
                theta_jn = 0, 
                psi = Psi,                       # polarisation
                phase = np.random.uniform(0,2*np.pi),                  
                geocent_time = 0,
                ra = Ra,                       
                dec = Dec, 
                #chi1 = 0 #spin for object 1 (but this is a number of chi_eff)
            )
     
    hx_gen, hp_gen, Hmod2, fs_gen = waveform(binary_parameters, waveform_arguments,Fp(Ra,Dec,Psi),Fx(Ra,Dec,Psi) )
                               
    snr   = SNR(fs_gen, Hmod2, psdinterp)
   
    omega = np.real( 8 /snr)
    
    return snr, omega, pwfunc(omega) #pwfunc takes care of when something goes out of range



Pdet = np.vectorize(Pdet) 

16:33 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_black_hole
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters


## For generating new random m and z
Normalisation constant cz and cm are the same regardless of Ns

(2.447036827279441, 5.91483085231679)

In [5]:
Ns = 100
A  = 1.6
B  = 3.4
Zp = 1 # z peak
Zm = 2 # z max


zgen, cz = z_distribution( sample_num = Ns, alpha=A, beta=B, z_p=Zp, zmax=Zm )
zgent    = np.tile( zgen,(Ns,1) )


m1gen,cm = m_distribution( sample_num = Ns, mmin = 5, mmax = 50, a = -2/3 )
m1gent   = np.tile( m1gen, (Ns,1) )
m1genT   = np.transpose( m1gent )

#np.save('DATA/Pdet_N'+str(Ns)+'rand.npy',PWfunc)
#np.save('DATA/mmatrixN'+str(Ns)+'_NoOrder.npy',m1genT)
#np.save('DATA/zmatrixN'+str(Ns)+'_NoOrder.npy',zgent)

cz,cm

(2.447036827279441, 5.91483085231679)

## For generating uniform m,z

In [27]:
Ns = 100
#print('Name: DATA/Pdet_N'+str(Ns)+'uniform.npy')

zuni = np.linspace(2/Ns,2,Ns)
muni = np.linspace(Mmin,Mmax,Ns)

m1uni,z1uni = tiling(muni,zuni,Ns)


array([0.0025, 0.005 , 0.0075, ..., 1.995 , 1.9975, 2.    ])

## Now run code for Pdet

In [ ]:
%%time
                    #Input:2d matrix, 2d mat, 2d matrix
Snr, Omega, PWfunc = Pdet( m1=m1uni, Z=z1uni, m2=m1genT, psdinterp=psdinterp3 ) 

### Saving data

In [63]:
np.save('DATA/Pdet_N'+str(Ns)+'.npy', PWfunc)
np.save('DATA/zmatrixN'+str(Ns)+'_NoOrder.npy',zgen)
np.save('DATA/mmatrixN'+str(Ns)+'_NoOrder.npy',m1gen)

G1 = np.load('DATA/Pdet_N'+str(Ns)+'.npy')

print('Filename: DATA/Pdet_N'+str(Ns)+'.npy,','shape=',np.shape(G1) )

Name: DATA/Pdet_N800uniform.npy


# Rate calculation functions
## 1. By integration ( for uniformly generated m and z )
## $ \int\int \frac{R(1+z)}{1+z} \frac{dV_c}{dz}P_{det}(z,m) p(m) dzdm $
Where
## $ p(z)\propto \frac{ (1+z)^ \alpha}{1+(\frac{1+z}{1+z_p})^{\alpha+\beta} }$
## $ p(m)\propto m^{(-2/3)} $

In [20]:
def integrand(x,m,Cm,Dvdz,pdet):
    frac   = (1+x)/(1+Zp)
    pz     = ((1+x)**A ) / (1+ frac**(A+B) ) 
    pm     = (m**(-2/3)) /Cm
    return R0 *  ( 1+(1+Zp)**(-A-B) ) * pz * Dvdz * pdet * pm /(1+x) 


def rate_int_uniform(pdet,Cm,mmin,mmax,zm,ns):
    zuni = np.linspace(0.1,zm,ns)
    muni = np.linspace(mmin,mmax,ns)
    MtileT, Ztile = tiling(muni,zuni,ns) 
    
    dvdz = 4*np.pi* cosmo.differential_comoving_volume(Ztile).value /(10**3)**3  #2d grid
    
    Int = integrand(Ztile, Cm, MtileT,dvdz,pdet) #pdet also in same grid


    # INTEGRATION STEPs
    dz = np.trapz( Int, Ztile, axis=1 )
   
    Mlist = MtileT[:,0]
    dNdt = np.trapz( dz, Mlist, axis=0 )  
    
    return dNdt   # this is dN/dt aka: rate



In [21]:
print('Rate for O3 with equal mass and Ns',1200,'=', rate_int_uniform( pdet12,cm,Mmin,Mmax,Zm,1200 ) )
print('Rate for O3 with equal mass and Ns',1000,'=', rate_int_uniform( pdet10,cm,Mmin,Mmax,Zm,1000 ) )
print('Rate for O3 with equal mass and Ns',800,'=', rate_int_uniform( pdet8,cm,Mmin,Mmax,Zm,800 ) )
print('Rate for O3 with equal mass and Ns',600,'=', rate_int_uniform( pdet6,cm,Mmin,Mmax,Zm,600 ) )
print('Rate for O3 with equal mass and Ns',400,'=', rate_int_uniform( pdet4,cm,Mmin,Mmax,Zm,400 ) )
print('Rate for O3 with equal mass and Ns',200,'=', rate_int_uniform( pdet2,cm,Mmin,Mmax,Zm,200 ) )


Rate for O3 with equal mass and Ns 1200 = 2.5034621153458376
Rate for O3 with equal mass and Ns 1000 = 2.503463474216751
Rate for O3 with equal mass and Ns 800 = 2.5034792934376653
Rate for O3 with equal mass and Ns 600 = 2.5035106735987647
Rate for O3 with equal mass and Ns 400 = 2.503611111402752
Rate for O3 with equal mass and Ns 200 = 2.5041929996443675


## 2. By using expectation values (for randomly generated m and z )

## $ \int f(z)p(z) dz  \approx  <f(z)>_{p(z)} $
## $ = \frac{1}{N_s^2} \sum_z { (R_0n \frac{c}{1+z} \frac{dV_c}{dz} P_{det})} $



In [22]:
def rate_exp( pdet, z, zp, Cz, ns ): 
    
    dvdz    = 4*np.pi* cosmo.differential_comoving_volume(z).value /(10**3)**3 
    formula = R0 * ( 1+(1+zp)**(-A-B) ) * Cz * dvdz * pdet /(1+z)
    
    return 1/ns/ns * sum(sum(formula))

In [24]:
print('rate as calculated from expectation value with Ns',1600,'=', rate_exp(pdet16ran,zgen16,Zp,cz,1600) )
print('rate as calculated from expectation value with Ns',1400,'=', rate_exp(pdet14ran,zgen14,Zp,cz,1400) )
print('rate as calculated from expectation value with Ns',1200,'=', rate_exp(pdet12ran,zgen12,Zp,cz,1200) )
#print('rate as calculated from expectation value with Ns',Ns,'=', rate_exp(pdet10ran,zgen10,2.447,1000) )
print('rate as calculated from expectation value with Ns',800,'=', rate_exp(pdet8ran,zgen8,Zp,cz,800) )

rate as calculated from expectation value with Ns 1600 = 8.597553231913414
rate as calculated from expectation value with Ns 1400 = 8.955470974352986
rate as calculated from expectation value with Ns 1200 = 10.013085359119998
rate as calculated from expectation value with Ns 800 = 8.392431301526344


In [ ]:

"""   
def rate_int(Ztile,Mtile,Mlist,pdet,ns):
    
    dvdz = 4*np.pi* cosmo.differential_comoving_volume(Ztile).value /(10**3)**3  #2d grid
    #ref points should be meshgrid

    
    Int = integrand(Ztile,Mtile,dvdz,pdet)

    # INTEGRATION STEP
    dz = np.trapz( Int, Ztile, axis=1 )
   
    dNdt = np.trapz( dz, Mlist, axis=0 )
    #dNdt = np.trapz( dm, Zlist, axis=0 )
    
    return dNdt   # this is dN/dt aka: rate

""" 

In [ ]:
#z is 2d and UNSORTED like pdet
"""
def rate_exp( pdet, z ): 
    
    frac   = (1+z)/(1+Zp)
    pz     = ((1+z)**A ) / (1+ frac**(A+B) ) 
    #print(z[0], 'all close?', np.allclose(z[:,0]) )
    #plt.pcolormesh(z)
    #plt.show()
    dvdz    = 4*np.pi* cosmo.differential_comoving_volume(z).value /(10**3)**3 
    formula = R0 * ( 1+(1+Zp)**(-A-B) ) * dvdz * pdet /(1+z)
    
    
    # each z has to be p(z) no??
    formula2 = R0 * ( 1+(1+Zp)**(-A-B) ) * dvdz * pdet /(1+pz)  # pdet ; 2d z, 2d m, not in order
    #plt.pcolormesh(mat)
    #plt.show()
    
    return formula2
    #return 1/ns * (sum(formula2)),  1/ns * (sum(formula)) #wrong
    #return 1/ns/ns * sum(sum(formula2)),  1/ns/ns * sum(sum(formula))


rate_exp = np.vectorize(rate_exp)  
#rate = rate_exp(z400t,Pdet400,z400,Ns)
rate = rate_exp(PWfunc,z14t)
print('rate as calculated from expectation value with Ns',1400,'=', rate )
"""

In [ ]:
# upload this on github
# /normalise m
# check if m distribution used in bayesian is consistent with ligo paper (fig10),fig(13)
# use continuity eqn

# run with. flat universe but without approx equations
# look at censor list

In [13]:
3*(50**(1/3)-5**(1/3))

5.922166655891068

In [ ]:
# note
#/ use mmin and zmin to find fmax - and use mmax and zmax to find fmin of the whole range

# / the rate formula is wrong for uniform dist.
# / change wavemode
# / check integral error estimation for rate